In [1]:
import setup_jwlab
import pandas as pd
import numpy as np

from jwlab.ml_prep import load_ml_df, y_to_binary
from sklearn.model_selection import cross_val_score

In [2]:
df = load_ml_df("~/projects/def-jwerker/kjslakov/data/ml_df_sktime.pkl")
df_2 = load_ml_df("~/projects/def-jwerker/kjslakov/data/ml_df_readys.pkl")

In [17]:
print(df.shape)
print(df_2.shape)

(653, 60)
(653, 60002)


In [4]:
# y = y_to_binary(df.label.values.flatten())
y = df_2.participant.values
y[y < 6] = 0
y[y >= 6] = 1
df = df.drop(columns=["label"], axis=1)

In [5]:
np.mean(y == 0)

0.6186830015313936

In [6]:
from scipy.stats import entropy, kurtosis, skew
funcs = [np.mean, np.min, np.max, np.var, skew, kurtosis]
dfs_applied = [df.applymap(f) for f in funcs]

In [7]:
from functools import reduce

suffix = 0
def join_dfs(a, b):
    global suffix
    suffix = suffix + 1
    return a.join(b, rsuffix="_%d" % suffix)

df_concat = reduce(join_dfs, dfs_applied)

In [8]:
df_concat.head()

,0,1,2,3,4,5,6,7,8,9,...,50_5,51_5,52_5,53_5,54_5,55_5,56_5,57_5,58_5,59_5
0,6.853466,0.348400,12.250030,23.596510,-1.851088,16.716675,21.320742,7.950176,19.992376,9.125198,...,-0.272569,0.206072,-0.382334,-0.990482,0.235168,0.997987,0.129002,0.659618,-0.441390,0.171510
1,-18.879231,-4.448306,1.029752,4.498039,-19.878193,-1.135712,7.389829,-7.171245,6.062628,-22.587683,...,-0.505275,-0.155466,-0.350456,-0.397194,-0.081773,-0.802797,-0.756513,-0.491387,-0.205341,-0.836175
2,13.288054,-14.234266,-16.283855,-17.121040,-11.281634,-9.797145,-23.788907,5.723077,-19.165705,3.194474,...,-1.065180,-0.954361,-0.093076,-1.284191,-0.406945,-0.711584,0.845993,-0.801256,-0.368082,0.281464
3,20.522480,-3.583972,2.115232,13.712213,10.745049,12.048008,15.790093,5.775336,9.180519,16.967114,...,0.085160,-0.273314,0.009561,0.023627,-0.948572,-0.951845,-1.008065,-0.995915,-0.460662,-0.687331
4,-11.072729,3.482658,13.058981,13.847923,-13.742619,9.967119,11.044140,6.476623,15.091089,-2.977680,...,-0.142354,-0.715358,-0.366312,-0.895248,-0.350184,-0.214313,-0.517301,0.527569,-0.270519,-0.388028


In [9]:
df_normalized = (df_concat - df_concat.mean()) / df_concat.std()
df_normalized.head()

,0,1,2,3,4,5,6,7,8,9,...,50_5,51_5,52_5,53_5,54_5,55_5,56_5,57_5,58_5,59_5
0,0.187910,-0.029600,0.869662,1.717514,-0.162854,1.066368,1.434642,0.355487,1.334462,0.273382,...,0.085211,0.890953,-0.089563,-1.027253,0.638892,2.414758,0.714436,1.789928,-0.346868,0.777242
1,-0.900831,-0.332462,-0.002406,0.214909,-1.067826,-0.258041,0.413386,-0.671108,0.272186,-1.122412,...,-0.193416,0.278766,-0.035308,-0.036481,0.287076,-0.925892,-0.851055,-0.258952,0.052960,-1.020129
2,0.460155,-0.950344,-1.348063,-1.486009,-0.636273,-0.900603,-1.872286,0.204289,-1.651713,0.012350,...,-0.863809,-1.073993,0.402742,-1.517739,-0.073876,-0.756683,1.981996,-0.810542,-0.222696,0.973361
3,0.766241,-0.277888,0.081960,0.939850,0.469480,0.720015,1.029198,0.207837,0.509955,0.618532,...,0.513532,0.079215,0.577426,0.666278,-0.675101,-1.202393,-1.295770,-1.157052,-0.379513,-0.754641
4,-0.570540,0.168296,0.932536,0.950528,-0.759816,0.565641,0.681278,0.255447,0.960692,-0.259308,...,0.241122,-0.669293,-0.062295,-0.868215,-0.010870,0.165810,-0.428153,1.554870,-0.057441,-0.220786


In [10]:
from sklearn.svm import LinearSVC
model = LinearSVC(max_iter=10000)

scores = cross_val_score(model, df_normalized, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.61832061 0.70229008 0.54198473 0.70992366 0.70542636]
Accuracy: 0.66 (+/- 0.13)


In [11]:
from sklearn.svm import SVC
model = SVC(gamma='scale')

scores = cross_val_score(model, df_concat, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.6259542  0.61832061 0.61832061 0.6870229  0.62015504]
Accuracy: 0.63 (+/- 0.05)


In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
scores = cross_val_score(model, df_concat, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/home/kjslakov/jupyter_py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/kjslakov/jupyter_py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/kjslakov/jupyter_py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.70229008 0.69465649 0.67175573 0.64885496 0.62790698]
Accuracy: 0.67 (+/- 0.06)


/home/kjslakov/jupyter_py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/kjslakov/jupyter_py3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
